In [130]:
import sys

sys.path.append("../../")
import pandas as pd
import numpy as np
import datetime
import os
from pprint import pprint
import matplotlib.pyplot as plt
import time
import vectorbt as vbt
from time import time

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 20)

plt.style.use("classic")
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams["axes.formatter.limits"] = [-1000000000, 1000000000]

In [131]:
I = {}
base_path = "../../phitech-data/01_raw"

for fname in os.listdir(base_path):
    I[fname.split("_")[0]] = pd.read_csv(f"{base_path}/{fname}")

I.keys()

dict_keys(['6B', 'MES'])

In [146]:
to_indicator = (
    vbt.IndicatorFactory(
        input_names=["indicator"],
        output_names=["values"],
    )
    .from_apply_func(lambda x: x)
    .run
)

fast = to_indicator(instruments["6B"].avg)
slow = to_indicator(instruments["6B"].avg_1)

entries = fast.values_crossed_below(slow.values)
exits = fast.values_crossed_above(slow.values)
exits.shape, entries.shape, instruments["6B"].shape

((58438,), (58438,), (58438, 15))

In [143]:
res = vbt.Portfolio.from_signals(instruments["6B"].close, entries=entries, exits=exits)
res.plot_value()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': '32a63939-14eb-459a-abd8-c1b5b2c5be10',
              'x': array([    0,     1,     2, ..., 58435, 58436, 58437]),
              'y': array([100., 100., 100., ..., 100., 100., 100.])},
             {'connectgaps': False,
              'fill': 'tonexty',
              'fillcolor': 'rgba(0, 128, 0, 0.3)',
              'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': '6cc49b30-b47f-471a-811f-bc5081e65f37',
              'x': array([    0,     1,     2, ..., 58435, 58436, 58437]),
              'y': array([100.        , 100.        , 100.        , ..., 101.98492758,
                          101.98492758, 101.